<a href="https://colab.research.google.com/github/mohammadZali/RegressionByCGAN/blob/main/AirRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd 
from sklearn import preprocessing
import os
from google.colab import drive
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from __future__ import print_function, division
import numpy as np
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam, SGD
import keras
from keras import Input, Model
from keras.layers import Dense, LeakyReLU, concatenate

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/project8/fAOD_PM_g3_55.csv')

In [ ]:
df['varPM']=df["PMc"]
df['varAOD'] = df["nAODm"] 

In [ ]:
X = df[['varAOD', 'lat', 'long', 'Prob_bestm', 'Prob_medm', 'month', 'd2m', 't2m','blh',
      'sp', 'lai_hv', 'lai_lv', 'ws10', 'wd10', 'cdir', 'uvb', 'RH', 'DOY']]

print(X.columns)
y = df[['varPM']]
y = y.to_numpy()

scaler = MinMaxScaler()
Xs = scaler.fit_transform(X)
print(Xs.shape)
print(y.shape)

cv = KFold(n_splits=10, random_state=1, shuffle=True)

i = 0
for train_index, test_index in cv.split(X):
  print("loop:", i)
  if i == 6:
    print( "TRAIN:", train_index, "TEST:", test_index)
    break
  else:
    i+=1
Xtrain = Xs[train_index]
ytrain = y[train_index]

Xtest = Xs[test_index]
ytest = y[test_index]

print(Xtrain.shape)
print(ytrain.shape)

Index(['varAOD', 'lat', 'long', 'Prob_bestm', 'Prob_medm', 'month', 'd2m',
       't2m', 'blh', 'sp', 'lai_hv', 'lai_lv', 'ws10', 'wd10', 'cdir', 'uvb',
       'RH', 'DOY'],
      dtype='object')
(19862, 18)
(19862, 1)
loop: 0
loop: 1
loop: 2
loop: 3
loop: 4
loop: 5
loop: 6
TRAIN: [    0     4     5 ... 19859 19860 19861] TEST: [    1     2     3 ... 19843 19846 19850]
(17876, 18)
(17876, 1)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(Xtrain, ytrain);



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [ ]:
dftest = df.loc[test_index]


# prediction on validation data
dftest["y_pred"] = rf.predict(Xtest)

def NominalPM(row):
    return row["y_pred"]/((1-row["RH"])**(-1))
dftest["PMp"] = dftest.apply(NominalPM, axis = 1)

print ("RMSE: ", np.round(mean_squared_error(dftest["PMp"], dftest["PM2.5"], squared=False),2))
print ("MAE: ", np.round(mean_absolute_error(dftest["PM2.5"], dftest["PMp"]),2))
from scipy.stats import pearsonr
corr, _ = pearsonr(dftest["y_pred"], dftest["PMc"])
print('Pearsons R2 correlation: %.3f' % corr**2)

RMSE:  8.42
MAE:  6.47
Pearsons R2 correlation: 0.779
